In [1]:
import sys
import os
import pandas as pd 
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.datasets.amazon_reviews import get_review_data
from recommenders.models.sasrec.ssept import SSEPT
from recommenders.models.sasrec.sampler import WarpSampler
from util_seren import SASRecDataSet
from recommenders.utils.notebook_utils import store_metadata
from recommenders.utils.timer import Timer

from model_seren import SASREC 

print(f"System version: {sys.version}")
print(f"Tensorflow version: {tf.__version__}")

System version: 3.9.19 (main, Mar 21 2024, 17:21:27) [MSC v.1916 64 bit (AMD64)]
Tensorflow version: 2.12.0


In [2]:
num_epochs = 10
batch_size = 128
seed = 100  # Set None for non-deterministic result

# data_dir = os.path.join("tests", "recsys_data", "RecSys", "SASRec-tf2", "data")
data_dir = os.path.join("./")

# Amazon Electronics Data
dataset = "reviews_Books_5"

lr = 0.0001  # learning rate
maxlen = 150  # maximum sequence length for each user
num_blocks = 3  # number of transformer blocks
hidden_units = 128  # number of units in the attention calculation
num_heads = 2  # number of attention heads
dropout_rate = 0.2  # dropout rate
l2_emb = 0.001  # L2 regularization coefficient (adjusted to 0.001)
num_neg_test = 20  # number of negative examples per positive example
model_name = 'sasrec'  # 'sasrec' or 'ssept'

In [3]:
inp_file = os.path.join(data_dir, dataset + ".txt")
print(inp_file)

# initiate a dataset class 
data = SASRecDataSet(filename=inp_file, col_sep="\t")

# create train, validation and test splits
data.split()

# some statistics
num_steps = int(len(data.user_train) / batch_size)
cc = 0.0
for u in data.user_train:
    cc += len(data.user_train[u])
print('%g Users and %g items' % (data.usernum, data.itemnum))
print('average sequence length: %.2f' % (cc / len(data.user_train)))

./reviews_Books_5.txt
477 Users and 8023 items
average sequence length: 23.43


In [4]:
model = SASREC(item_num=data.itemnum,
               seq_max_len=maxlen,
               num_blocks=num_blocks,
               embedding_dim=hidden_units,
               attention_dim=hidden_units,
               attention_num_heads=num_heads,
               dropout_rate=dropout_rate,
               conv_dims = [128, 128],
               l2_reg=l2_emb,
               num_neg_test=num_neg_test
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
model.compile(optimizer=optimizer)

In [5]:
sampler = WarpSampler(data.user_train, data.usernum, data.itemnum, batch_size=batch_size, maxlen=maxlen, n_workers=3)


In [6]:
with Timer() as train_time:
    results = model.train(data, sampler, num_epochs=num_epochs, batch_size=batch_size, lr=lr, val_epoch=10)

print('Time cost for training is {0:.2f} mins'.format(train_time.interval/60.0))

Evaluating...



epoch: 10, time: 31.6305741, valid: {'HT': {5: 0.2194513715710723, 10: 0.3740648379052369}, 'NDCG': {5: 0.1452883917793154, 10: 0.19499106050741455}, 'HRseren': {5: 0.35294117647058826, 10: 0.47058823529411764}, 'NDCGseren': {5: 0.23090301793552098, 10: 0.2706653561904578}})
epoch: 10, time: 31.6305741,  test: {'HT': {5: 0.20448877805486285, 10: 0.3491271820448878}, 'NDCG': {5: 0.13453178233227422, 10: 0.18068590493865896}, 'HRseren': {5: 0.14285714285714285, 10: 0.3142857142857143}, 'NDCGseren': {5: 0.0862223588801714, 10: 0.13847915471110858}})


Results:
HT@5: 0.20448877805486285
HT@10: 0.34413965087281795
NDCG@5: 0.13680388537582902
NDCG@10: 0.18128716603501868
HRseren@5: 0.14285714285714285
HRseren@10: 0.34285714285714286
NDCGseren@5: 0.09405799069200511
NDCGseren@10: 0.16074126623180052
Time cost for training is 1.30 mins


In [7]:
print(results)

{'HT': {5: 0.20448877805486285, 10: 0.34413965087281795}, 'NDCG': {5: 0.13680388537582902, 10: 0.18128716603501868}, 'HRseren': {5: 0.14285714285714285, 10: 0.34285714285714286}, 'NDCGseren': {5: 0.09405799069200511, 10: 0.16074126623180052}}


In [8]:
model.evaluate_serendipity(dataset=data)

  0%|                               | 1/477 [00:00<01:04,  7.41user/s]

user 1 processed. 80 interactions.
user 2 processed. 7 interactions.
user 3 processed. 10 interactions.
user 4 processed. 4 interactions.
user 5 processed. 22 interactions.
user 6 processed. 3 interactions.


  3%|▊                             | 12/477 [00:00<00:08, 54.35user/s]

user 7 processed. 18 interactions.
user 8 processed. 4 interactions.
user 9 processed. 3 interactions.
user 10 processed. 21 interactions.
user 11 processed. 2 interactions.
user 12 processed. 38 interactions.
user 13 processed. 1 interactions.
user 14 processed. 15 interactions.


  5%|█▍                            | 23/477 [00:00<00:10, 41.68user/s]

...

 97%|████████████████████████████▎| 465/477 [00:13<00:00, 31.56user/s]

user 460 processed. 152 interactions.
user 461 processed. 4 interactions.
user 462 processed. 5 interactions.
user 463 processed. 3 interactions.
user 464 processed. 101 interactions.
user 465 processed. 3 interactions.


 99%|████████████████████████████▌| 470/477 [00:13<00:00, 30.89user/s]

user 466 processed. 111 interactions.
user 467 processed. 17 interactions.
user 468 processed. 5 interactions.
user 469 processed. 3 interactions.
user 470 processed. 28 interactions.
user 471 processed. 2 interactions.
user 472 processed. 5 interactions.
user 473 processed. 5 interactions.


user 474 processed. 125 interactions.
user 475 processed. 15 interactions.
user 476 processed. 3 interactions.
user 477 processed. 15 interactions.
Serendipity comparison saved
